
#### NCSU GIS 714: Geospatial Computation and Simulations

## Data simulation: Part 2

Contents:
* Start GRASS GIS
* Random Fractal Surfaces
* Random Point Data

Additional Resources:
* [GRASS GIS overview and manual](http://grass.osgeo.org/grass74/manuals/index.html)
* [GRASS book](http://www.grassbook.org/)

#### Start GRASS GIS

In [ ]:
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
# FOR WINDOWS:
# grass_call = "grass80"
# shell_setting = True
# FOR MAC/LINUX
grass_call = "grass"
shell_setting = False

sys.path.append(
    subprocess.check_output([grass_call, "--config", "python_path"], text=True, shell=shell_setting).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("../data", "nc_spm_08_grass7", "HW2_data_simulation")

# Set Region
gs.run_command("g.region", region="rural_1m")

### Random fractal surfaces

First, we generate two series of surfaces with dimension 2.9 to 2.01. The series shows creation of a fractal surface, each step with increasing complexity. We register the series in temporal framework, assign the rasters a common color table and create an animation
(feel free to select larger n or a different dimension). The two text files, *fractal_dem290_series.txt* and *fractal_dem201_series.txt*, contain a list of names for each in the series and the time step. Finally, we'll run watershed analysis on the final surfaces.

#### *Question*
Create a fractal surface with [r.surf.fractal](https://grass.osgeo.org/grass80/manuals/r.surf.fractal.html) to create a raster named `fractal_dem_d290` with dimension 2.9 and 9 intermediate images. Use [r.info](https://grass.osgeo.org/grass80/manuals/r.info.html) to find the minimun and maximum values of the surface. Describe your workflow.

In [ ]:
### YOUR CODE AND TEXT HERE

Now, we'll register each of the intermediate images as a space time raster dataset and compute a few simple statistics.

In [ ]:
gs.run_command("t.create", output="fractal_dem_290", type="strds", temporaltype="relative", 
               title="Fractal DEMs with d=2.90", description="Generated data")

# Register series with names from file
gs.run_command("t.register", input="fractal_dem_290", type="raster", file="fractal_dem290_series.txt", unit="years")

print(gs.read_command("t.rast.list", input="fractal_dem_290"))

Create an animation showing the creation of the fractal surface:

In [ ]:
fractal_creation = gj.TimeSeriesMap(use_region=True)
fractal_creation.add_raster_series("fractal_dem_290")
fractal_creation.show()

Get some simple univariate statistics

In [ ]:
print(gs.read_command("t.rast.univar", input="fractal_dem_290"))

#### *Question*
Repeat the process above but for dimension = 2.01. Compute and compare the flow accumulations and drainage basins for both dimensions (2.9 and 2.01). How are they different?

In [ ]:
#### YOUR CODE AND TEXT HERE.

### Surfaces with Fractal Noise

#### *Question*
Let's return to the DEMs with stocastic noise that we created in part 1.

Say the lidar error is +/- 30 cm. Rescale the fractal surface to [-0.3,0.3] and add to the `elev_lid792_1m` DEM as a simulated noise. Also compute the flow accumation, drainage direction and basins with a threshold of 15000 (similar to part 1). 

The water flow pattern may represent flow over rough surface, where the roughness is represented in the DEM (e.g. as a grass cover) rather than a constant value (e.g. Mannings coefficient).

In [ ]:
#### YOUR CODE AND TEXT HERE

### Comparing stochastic noise to real LiDAR sensor noise

In addition to the lidar-derived, bare-earth DEM, we have the original lidar point data, `elev_lid792_bepts`. Next, let's compute the elevation surface from point data with different tension parameters and compare the impact on flow modeling.

In [ ]:
gs.run_command("g.region", raster="elev_lid792_1m", flags="p")

# First, use the default tension (40)
gs.run_command("v.surf.rst", input="elev_lid792_bepts", elev="elev_lid_default", npmin=120) 

# Higher Tension
gs.run_command("v.surf.rst", input="elev_lid792_bepts", elev="elev_lid_t120", npmin=120, ten=120)

# Compute flow accumulation, drainage direction and basins for comparison to DEMs with stochastic noise
gs.run_command("r.watershed", elevation="elev_lid_default", threshold=5000, accumulation="accum_5K_liddef", drainage="draindir_5K_liddef",
               basin="basin_5K_liddef", flags="a")

gs.run_command("r.watershed", elevation="elev_lid_t120", threshold=5000, accumulation="accum_5K_lidt120", drainage="draindir_5K_lidt120",
               basin="basin_5K_lidt120", flags="a")

#### *Question*

Compare the drainage patterns. Which of the surfaces with simulated noise (uniform, gaussian, spatially dependent gaussian and fractal) produce the flow pattern closest to the surface with noise from lidar point cloud (ten=120)? 

In [ ]:
#### YOUR CODE AND TEXT HERE

###  Random point data 

In this example, we analyze how errors in point position can influence viewshed analysis. First, install r.viewshed.cva addon:

In [ ]:
!g.extension r.viewshed.cva

Generate random points and perturb them. For both points set compute a cumulative viewshed and compare them:

In [ ]:
# Generate random points and compute a cumulative viewshed
gs.run_command("v.random", output="random_view", npoints=10, seed=4)
gs.run_command("r.viewshed.cva", input="elev_lid792_1m", vector="random_view", output="cumulative")

#### *Question*

Perturb the points with [v.perturb](https://grass.osgeo.org/grass80/manuals/v.perturb.html) with parameters = 20 and seed = 1. Computer the cummulative viewshed for the perturbed point and compare the results (in addition to mapping the viewsheds, [r.report](https://grass.osgeo.org/grass80/manuals/r.report.html) may be useful). What is the impact of error in point position on total viewshed area? 

In [ ]:
#### YOUR CODE AND TEXT HERE

Additional commands you can use to simulate simulate point data in raster or vector representation:
r.random, r.random.cells, r.sample.category, v.perturb, v.random, v.kcv 

### Additional Information
* [Website](https://github.com/ncsu-geoforall-lab/geospatial-simulations-course)
    
* [Computing Help](https://help.ncsu.edu/)
    
* [GIST Home](https://geospatial.ncsu.edu/)
    
* [Disclaimer](https://www.ncsu.edu/policies/prr-disclaimer.php)
    
* [Accessibility](https://oit.ncsu.edu/itaccess)

* License: 2018 [CC BY-SA](https://creativecommons.org/licenses/by-sa/4.0/)
  
* [NCSU GeoForAll Lab](https://geospatial.ncsu.edu/geoforall/)